In [1]:
import pandas as pd 
import numpy as np 
from uszipcode import SearchEngine
search = SearchEngine(simple_zipcode=True)

In [2]:
all_zip = pd.read_csv('./all_zipcodes.csv')

In [3]:
all_zip.fillna(0, inplace=True)

In [4]:
#Small function that makes output damage cost easier to read, Basic premise is to format the string
#split it by commas, then grab the first integer block based on how many blocks we can add millions/billions 
#or even trillions of dollars. it then returns 75%-100% of the total to give a wide estimate. This function is 
#called only in the main search function below
def function_total(total_damage):
    #converting to integer
    convert = int(total_damage)
#     formating in commas
    formater = format(convert, ',')
#series of if statements to decide whether to use millions/billions ect.
    if len(formater.split(',')) == 3:
        print(f'Estimated damage between {int(formater[0]) * .75} and {formater[0]} Million dollars')
    elif len(formater.split(',')) == 2:
        print(f'Estimated damage between {int(formater[0]) * .75} and {formater[0]} Thousand dollars') 
    elif len(formater.split(',')) == 4:
        print(f'Estimated damage between {int(formater[0]) * .75} and {formater[0]} Billion dollars')  
    elif total_damage >= 1_000_000_000_000:
        print(f'Estimated damage over 1 trillion dollars')
    else: print(f'Estimated damage: {total_damage * .75}-{total_damage} dollars') 

In [5]:
#main function that will take the epicenter for disaster, calculate area of affect, find all zipcodes in that area
#and calculate a projected damage cost based on median home value and number of homes in area. with a penalty reduction
#assuming not all homes were destroyed.
#entering damage_radius and percent are optional, The code will revert to standard values based on previous disaster type
def main_search(Zip, disaster_type, damage_radius=None, damage_percent=None):
    try:
    #setting dictionary for area covered by disaster, in KM   
        disaster_dist = {
            'hurricane': 100,
            'flood': 50,
            'fire':.5
        }
    #setting the effect ratio based on research from last 5 disasters by type, 
    #(More info needed for more accurate calculations)
        disaster_effect = {
            'hurricane': .1,
            'flood' : .81,
            'fire': 1
        }
        if damage_radius == None:
            radius = disaster_dist[disaster_type]
        else:
            radius = damage_radius
        if damage_percent == None:
            damage = disaster_effect[disaster_type]
        else:
            damage = damage_percent
            
        #instantiate the search engine
        search = SearchEngine(simple_zipcode=True)
        #searching for epicenter zipcode
        zipcode = search.by_zipcode(str(Zip))
        #Setting result as a list with every zipcode within a certain range. 
        result = search.by_coordinates(zipcode.lat,zipcode.lng, radius= radius, returns=25)
        #printing the median household value for epicenter zipcode 
        print(f'The median house value of {Zip} ({zipcode.major_city}) is: {zipcode.median_home_value}')
        #iterate through our results and getting median home value times number of housing units in each zipcode effected by hurricane 
        value = 0
        
        for i in result:
            zipcode = i.zipcode
            try: 
            #Try and except statements to account for null values where no houses exist.
                #totaling up the maximum possible damage 
                value += all_zip['median_house_value'][int(zipcode)] * all_zip['housing_units'][int(zipcode)]
                
            except:
                pass
        #Printing the total damage done minus the penalty for the specific disaster/input of percent of homes damaged. 
        total_value = value * damage
        #
        function_total(total_value)
        
    except KeyError:
        print('Zipcode was not found/ was invalid')

In [6]:
#test
main_search(21791, 'hurricane', )

The median house value of 21791 (Union Bridge) is: 319800
Estimated damage between 0.75 and 1 Billion dollars
